# 0. Set Up

In [1]:
# import libraries
import pandas as pd 
import pickle as pk
import numpy as np
import os
import openpyxl

In [3]:
# Reads each csv file and converts it into a 2-D data structure (Dataframe).

# Get root folder for reading data
data_fp = os.getcwd().replace('data_management/merge', 'charter_data/ccd/')

# Directory
f0 = pd.read_csv(data_fp + "ccd_201718/ccd_sch_029_1718_w_1a_083118.csv", encoding = "latin1", low_memory = False)
#f0 = pd.read_csv("ccd_2017-18/Directory1718.csv", encoding = "latin1", low_memory = False)

# Lunch Program
f1 = pd.read_csv(data_fp + "ccd_201718/ccd_sch_033_1718_l_1a_083118.csv", encoding = 'latin1', low_memory = False)
#f1 = pd.read_csv("ccd_2017-18/Lunch2017-18.csv", encoding = 'latin1', low_memory = False)

# Membership - Large file (2.4GB), may need to extra file first
f2 = pd.read_csv(data_fp + "ccd_201718/ccd_SCH_052_1718_l_1a_083118.csv", encoding = 'latin1', low_memory = False)
#f2 = pd.read_csv("ccd_2017-18/ccd_SCH_052_1718_l_1a_083118.csv", encoding = 'latin1', low_memory = False)

# Staff
f3 = pd.read_csv(data_fp + "ccd_201718/ccd_sch_059_1718_l_1a_083118.csv", encoding = 'latin1', low_memory = False)
#f3 = pd.read_csv("ccd_2017-18/staff17-18.csv", encoding = 'latin1', low_memory = False)

# School Characteristics
f4 = pd.read_csv(data_fp + "ccd_201718/ccd_sch_129_1718_w_1a_083118.csv", encoding = 'latin1', low_memory = False)
#f4 = pd.read_csv("ccd_2017-18/SchoolCharacteristics2017-18.csv", encoding = 'latin1', low_memory = False)

# School Geography
fgeo = pd.read_excel(data_fp + "ccd_201718/EDGE_GEOCODE_PUBLICSCH_1718/EDGE_GEOCODE_PUBLICSCH_1718.xlsx") #, encoding = 'latin1')

In [4]:
# pd.set_option is to display all columns when calling dataframe
pd.set_option('display.max_columns', None)
# display the number of columns and rows of each data table
display(np.shape(f0))
display(np.shape(f1))
display(np.shape(f2))
display(np.shape(f3))
display(np.shape(f4))
display(np.shape(fgeo))

(102337, 65)

(494745, 17)

(12112911, 18)

(99899, 15)

(99899, 20)

(102337, 24)

# 1. Select Free and Reduced Lunch from f1 table

In [5]:
f1["LUNCH_PROGRAM"].unique()

array(['Free lunch qualified', 'Reduced-price lunch qualified', 'Missing',
       'Not Applicable', 'No Category Codes'], dtype=object)

In [6]:
lunch_labels = ['Free lunch qualified', 'Reduced-price lunch qualified']
f1_filtered = f1.loc[f1['LUNCH_PROGRAM'].isin(lunch_labels) , : ]

In [7]:
f1_wide = f1_filtered.groupby("NCESSCH").agg({"STUDENT_COUNT": ['sum']})

In [8]:
f1_wide.columns = ['Free and Reduced Lunch']

# 2. Explore Membership Table - f2

In [10]:
f2["TOTAL_INDICATOR"].unique()

array(['Category Set A - By Race/Ethnicity; Sex; Grade',
       'Derived - Education Unit Total minus Adult Education Count',
       'Derived - Subtotal by Race/Ethnicity and Sex minus Adult Education Count',
       'Education Unit Total', 'Subtotal 4 - By Grade'], dtype=object)

In [11]:
# Figure out what each of these indicators mean.
f2notNan = f2.loc[pd.notnull(f2['STUDENT_COUNT']), ]

In [12]:
# specific school membership
totals_id = ["Category Set A - By Race/Ethnicity; Sex; Grade"]
f2one = f2.loc[f2["NCESSCH"] == 780003000034, ]
f2onefilter = f2one.loc[f2['TOTAL_INDICATOR'].isin(totals_id) , : ]
f2onesum = f2onefilter.agg({"STUDENT_COUNT": ['sum']})
f2onesum

,STUDENT_COUNT
sum,504.0


In [13]:
f2one.loc[f2['TOTAL_INDICATOR'] == "Education Unit Total"]

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,GRADE,RACE_ETHNICITY,SEX,STUDENT_COUNT,TOTAL_INDICATOR,DMS_FLAG
12112906,2017-2018,78,U.S. VIRGIN ISLANDS,VI,BERTHA BOSCHULTE JUNIOR HIGH,1,NaN,VI-001,7800030,VI-001-9,780003000034,7800034,No Category Codes,No Category Codes,No Category Codes,504.0,Education Unit Total,Reported


# 3. Extract total enrollment of student of each school

In [14]:
total_member = f2.loc[f2["TOTAL_INDICATOR"] == "Education Unit Total"]
total_member

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,GRADE,RACE_ETHNICITY,SEX,STUDENT_COUNT,TOTAL_INDICATOR,DMS_FLAG
227,2017-2018,1,ALABAMA,AL,Albertville Middle School,1,NaN,AL-101,100005,AL-101-0010,10000500870,100870,No Category Codes,No Category Codes,No Category Codes,807.0,Education Unit Total,Reported
470,2017-2018,1,ALABAMA,AL,Albertville High School,1,NaN,AL-101,100005,AL-101-0020,10000500871,100871,No Category Codes,No Category Codes,No Category Codes,1436.0,Education Unit Total,Reported
713,2017-2018,1,ALABAMA,AL,Evans Elementary School,1,NaN,AL-101,100005,AL-101-0110,10000500879,100879,No Category Codes,No Category Codes,No Category Codes,850.0,Education Unit Total,Reported
956,2017-2018,1,ALABAMA,AL,Albertville Elementary School,1,NaN,AL-101,100005,AL-101-0200,10000500889,100889,No Category Codes,No Category Codes,No Category Codes,906.0,Education Unit Total,Reported
1199,2017-2018,1,ALABAMA,AL,Big Spring Lake Kindergarten School,1,NaN,AL-101,100005,AL-101-0035,10000501616,101616,No Category Codes,No Category Codes,No Category Codes,418.0,Education Unit Total,Reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12112522,2017-2018,78,U.S. VIRGIN ISLANDS,VI,GLADYS A. ABRAHAM ELEMENTARY SCHOOL,1,NaN,VI-001,7800030,VI-001-21,780003000025,7800025,No Category Codes,No Category Codes,No Category Codes,279.0,Education Unit Total,Reported
12112653,2017-2018,78,U.S. VIRGIN ISLANDS,VI,ULLA F MULLER ELEMENTARY SCHOOL,1,NaN,VI-001,7800030,VI-001-17,780003000026,7800026,No Category Codes,No Category Codes,No Category Codes,334.0,Education Unit Total,Reported
12112784,2017-2018,78,U.S. VIRGIN ISLANDS,VI,YVONNE BOWSKY ELEMENTARY SCHOOL,1,NaN,VI-001,7800030,VI-001-23,780003000027,7800027,No Category Codes,No Category Codes,No Category Codes,276.0,Education Unit Total,Reported
12112840,2017-2018,78,U.S. VIRGIN ISLANDS,VI,CANCRYN JUNIOR HIGH SCHOOL,1,NaN,VI-001,7800030,VI-001-25,780003000033,7800033,No Category Codes,No Category Codes,No Category Codes,433.0,Education Unit Total,Reported


In [15]:
# calculate the total of each school
total_mem = f2.loc[f2["TOTAL_INDICATOR"] == "Education Unit Total",['NCESSCH','STUDENT_COUNT']]
total_mem

,NCESSCH,STUDENT_COUNT
227,10000500870,807.0
470,10000500871,1436.0
713,10000500879,850.0
956,10000500889,906.0
1199,10000501616,418.0
...,...,...
12112522,780003000025,279.0
12112653,780003000026,334.0
12112784,780003000027,276.0
12112840,780003000033,433.0


# 4. Calculate student total by race

In [16]:
# Total up all students by race for each school
sch_memfilter = f2.loc[f2['TOTAL_INDICATOR'].isin(totals_id) , : ]
sch_memgroup = sch_memfilter.groupby(["NCESSCH", 'RACE_ETHNICITY']).agg({"STUDENT_COUNT": ['sum']})
sch_memgroup

STUDENT_COUNT
                                                                 sum
NCESSCH      RACE_ETHNICITY                                         
10000500870  American Indian or Alaska Native                    0.0
             Asian                                               5.0
             Black or African American                          32.0
             Hispanic/Latino                                   368.0
             Native Hawaiian or Other Pacific Islander           1.0
...                                                              ...
780003000034 Hispanic/Latino                                    39.0
             Native Hawaiian or Other Pacific Islander           0.0
             Not Specified                                       0.0
             Two or more races                                   0.0
             White                                               7.0

[790184 rows x 1 columns]

In [17]:
sch_memwide = sch_memgroup.unstack()

In [18]:
sch_memwide.index

Int64Index([ 10000500870,  10000500871,  10000500879,  10000500889,
             10000501616,  10000502150,  10000600193,  10000600872,
             10000600876,  10000600877,
            ...
            780003000018, 780003000019, 780003000020, 780003000022,
            780003000024, 780003000025, 780003000026, 780003000027,
            780003000033, 780003000034],
           dtype='int64', name='NCESSCH', length=98773)

In [19]:
schMemLun = sch_memwide.merge(f1_wide, how="outer", left_index = True, right_index = True)
schMemLunTot = schMemLun.merge(total_mem, how="outer", left_index = True, right_on = "NCESSCH")

/opt/conda/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (3 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [20]:
sch_merged = schMemLunTot.set_index('NCESSCH')
sch_merged

,"(STUDENT_COUNT, sum, American Indian or Alaska Native)","(STUDENT_COUNT, sum, Asian)","(STUDENT_COUNT, sum, Black or African American)","(STUDENT_COUNT, sum, Hispanic/Latino)","(STUDENT_COUNT, sum, Native Hawaiian or Other Pacific Islander)","(STUDENT_COUNT, sum, Not Specified)","(STUDENT_COUNT, sum, Two or more races)","(STUDENT_COUNT, sum, White)",Free and Reduced Lunch,STUDENT_COUNT
NCESSCH,,,,,,,,,,
10000500870,0.0,5.0,32.0,368.0,1.0,0.0,18.0,383.0,570.0,807.0
10000500871,4.0,8.0,53.0,614.0,0.0,0.0,41.0,716.0,832.0,1436.0
10000500879,1.0,5.0,24.0,436.0,0.0,0.0,20.0,364.0,608.0,850.0
10000500889,4.0,2.0,34.0,443.0,1.0,0.0,33.0,389.0,684.0,906.0
10000501616,0.0,3.0,14.0,224.0,1.0,0.0,12.0,164.0,345.0,418.0
...,...,...,...,...,...,...,...,...,...,...
780003000025,0.0,1.0,170.0,103.0,1.0,0.0,1.0,3.0,276.0,279.0
780003000026,1.0,4.0,297.0,26.0,0.0,0.0,0.0,6.0,331.0,334.0
780003000027,1.0,5.0,260.0,9.0,0.0,0.0,0.0,1.0,273.0,276.0


In [21]:
proportions = sch_merged.copy()
columns = proportions.columns
for i, row in proportions.iterrows():
    for x in range(len(columns)):
        col = columns[x]
        if col != 'STUDENT_COUNT':
            value = row[x]
            if value == 0:
                proportions.at[i, col] = 0
            else:
                proportions.at[i, col] = str((row[x] / row[len(row) - 1]) * 100)

proportions.columns = ['American Indian/Alaska Native (% total)', 'Asian (% total)', 'Black/African American (% total)', 'Hispanic/Latino (% total)', 'Hawaiian/Pacific Islander (% total)', 'Not Specified (% total)', 'Two+ races (% total)', 'White (% total)', 'Free or Reduced Lunch (% total)', 'Total Student Count']              
proportions

<ipython-input-21-4a2722ef57bc>:11: RuntimeWarning: divide by zero encountered in double_scalars
  proportions.at[i, col] = str((row[x] / row[len(row) - 1]) * 100)


,American Indian/Alaska Native (% total),Asian (% total),Black/African American (% total),Hispanic/Latino (% total),Hawaiian/Pacific Islander (% total),Not Specified (% total),Two+ races (% total),White (% total),Free or Reduced Lunch (% total),Total Student Count
NCESSCH,,,,,,,,,,
10000500870,0.000000,0.619579,3.965304,45.600991,0.123916,0.0,2.230483,47.459727,70.631970,807.0
10000500871,0.278552,0.557103,3.690808,42.757660,0.000000,0.0,2.855153,49.860724,57.938719,1436.0
10000500879,0.117647,0.588235,2.823529,51.294118,0.000000,0.0,2.352941,42.823529,71.529412,850.0
10000500889,0.441501,0.220751,3.752759,48.896247,0.110375,0.0,3.642384,42.935982,75.496689,906.0
10000501616,0.000000,0.717703,3.349282,53.588517,0.239234,0.0,2.870813,39.234450,82.535885,418.0
...,...,...,...,...,...,...,...,...,...,...
780003000025,0.000000,0.358423,60.931900,36.917563,0.358423,0.0,0.358423,1.075269,98.924731,279.0
780003000026,0.299401,1.197605,88.922156,7.784431,0.000000,0.0,0.000000,1.796407,99.101796,334.0
780003000027,0.362319,1.811594,94.202899,3.260870,0.000000,0.0,0.000000,0.362319,98.913043,276.0


# 5. Merge all Tables

In [22]:
#Merge All tables
merged_201718 = f0.merge(proportions, how="outer", left_on = "NCESSCH", right_index = True)

In [23]:
merged_201718 = merged_201718.merge(f3, how="outer", left_on = "NCESSCH", right_on = "NCESSCH")

In [24]:
merged_201718 = merged_201718.merge(f4, how="outer", left_on = "NCESSCH", right_on = "NCESSCH")

In [25]:
merged_201718 = merged_201718.merge(fgeo, how="outer", left_on = "NCESSCH", right_on = "NCESSCH")

In [26]:
merged_201718

,SCHOOL_YEAR_x,FIPST_x,STATENAME_x,ST_x,SCH_NAME_x,LEA_NAME,STATE_AGENCY_NO_x,UNION_x,ST_LEAID_x,LEAID_x,ST_SCHID_x,NCESSCH,SCHID_x,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,American Indian/Alaska Native (% total),Asian (% total),Black/African American (% total),Hispanic/Latino (% total),Hawaiian/Pacific Islander (% total),Not Specified (% total),Two+ races (% total),White (% total),Free or Reduced Lunch (% total),Total Student Count,SCHOOL_YEAR_y,FIPST_y,STATENAME_y,ST_y,SCH_NAME_y,STATE_AGENCY_NO_y,UNION_y,ST_LEAID_y,LEAID_y,ST_SCHID_y,SCHID_y,TEACHERS,TOTAL_INDICATOR,DMS_FLAG,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,SCHID,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR
0,2017-2018,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,NaN,AL-101,100005,AL-101-0010,10000500870,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336.0,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,NaN,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,04/04/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,07,08,Middle,As reported,0.000000,0.619579,3.965304,45.600991,0.123916,0.0,2.230483,47.459727,70.631970,807.0,2017-2018,1.0,ALABAMA,AL,Albertville Middle School,1.0,NaN,AL-101,100005.0,AL-101-0010,100870.0,36.0,Education Unit Total,Reported,2017-2018,1.0,ALABAMA,AL,Albertville Middle School,1.0,NaN,AL-101,100005.0,AL-101-0010,100870.0,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.260194,-86.206174,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,104,01026,01009,2017-2018
1,2017-2018,1,ALABAMA,AL,Albertville High School,Albertville City,1,NaN,AL-101,100005,AL-101-0020,10000500871,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399.0,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322.0,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,04/04/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported,0.278552,0.557103,3.690808,42.757660,0.000000,0.0,2.855153,49.860724,57.938719,1436.0,2017-2018,1.0,ALABAMA,AL,Albertville High School,1.0,NaN,AL-101,100005.0,AL-101-0020,100871.0,77.0,Education Unit Total,Reported,2017-2018,1.0,ALABAMA,AL,Albertville High School,1.0,NaN,AL-101,100005.0,AL-101-0020,100871.0,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.262226,-86.204884,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,104,01026,01009,2017-2018
2,2017-2018,1,ALABAMA,AL,Evans Elementary School,Albertville City,1,NaN,AL-101,100005,AL-101-0110,10000500879,100879,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300.0,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1346.0,(256)878-7698,http://www.albertk12.org,1,Open,1,Open,04/04/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,05,06,Middle,As reported,0.117647,0.588235,2.823529,51.294118,0.000000,0.0,2

In [27]:
#Perform check on full merged table
merged_201718.loc[merged_201718["SCH_NAME_x"] == "Aquarian Charter School", ]

,SCHOOL_YEAR_x,FIPST_x,STATENAME_x,ST_x,SCH_NAME_x,LEA_NAME,STATE_AGENCY_NO_x,UNION_x,ST_LEAID_x,LEAID_x,ST_SCHID_x,NCESSCH,SCHID_x,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,American Indian/Alaska Native (% total),Asian (% total),Black/African American (% total),Hispanic/Latino (% total),Hawaiian/Pacific Islander (% total),Not Specified (% total),Two+ races (% total),White (% total),Free or Reduced Lunch (% total),Total Student Count,SCHOOL_YEAR_y,FIPST_y,STATENAME_y,ST_y,SCH_NAME_y,STATE_AGENCY_NO_y,UNION_y,ST_LEAID_y,LEAID_y,ST_SCHID_y,SCHID_y,TEACHERS,TOTAL_INDICATOR,DMS_FLAG,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,SCHID,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR
1659,2017-2018,2,ALASKA,AK,Aquarian Charter School,Anchorage School District,1,NaN,AK-05,200180,AK-05-059010,20018000172,200172,1705 W 32nd Ave,NaN,NaN,Anchorage,AK,99517,2002.0,1705 W 32nd Ave,NaN,NaN,Anchorage,AK,99517,NaN,(907)742-4900,http://aquarian.asdk12.org/,1,Open,1,Open,02/20/2018,Regular School,1,No,No,Yes,0200180,Anchorage School District,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,KG,06,Elementary,As reported,2.597403,2.857143,1.298701,7.272727,0.25974,0.0,14.285714,71.428571,0.0,385.0,2017-2018,2.0,ALASKA,AK,Aquarian Charter School,1.0,NaN,AK-05,200180.0,AK-05-059010,200172.0,22.13,Education Unit Total,Reported,2017-2018,2.0,ALASKA,AK,Aquarian Charter School,1.0,NaN,AK-05,200180.0,AK-05-059010,200172.0,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,Aquarian Charter School,2,1705 W 32nd Ave,Anchorage,AK,99517,2,2020,Anchorage Municipality,11,61.192407,-149.916872,11260,"Anchorage, AK",1,N,N,N,N,200,02018,0200I,2017-2018


# 6. Drop redundant columns and clean full merged table

In [28]:
# Drop Repetitive Columns
merged_201718 = merged_201718.drop(['SCHOOL_YEAR_y', 'FIPST_y', 'STATENAME_y', 'ST_y', 'SCH_NAME_y', 'STATE_AGENCY_NO_y', 'UNION_y', 'ST_LEAID_y','LEAID_y', 'ST_SCHID_y','SCHID_y'
                                    ,'SCHOOL_YEAR', 'FIPST', 'STATENAME', 'ST', 'SCH_NAME', 'STATE_AGENCY_NO', 'UNION', 'ST_LEAID','LEAID', 'ST_SCHID','SCHID', ], axis=1)

In [29]:
merged_201718 = merged_201718.rename(columns={'SCHOOL_YEAR_x':'SCHOOL_YEAR',
                                              'FIPST_x' : 'FIPST',
                                              'STATENAME_x' : 'STATENAME', 
                                              'ST_x' : 'ST',
                                              'SCH_NAME_x' : 'SCH_NAME',
                                              'STATE_AGENCY_NO_x' : 'STATE_AGENCY_NO',
                                              'UNION_x' : 'UNION',
                                              'ST_LEAID_x' : 'ST_LEAID',
                                              'LEAID_x': 'LEAID',
                                              'ST_SCHID_x' : 'ST_SCHID' ,
                                              'SCHID_x' : 'SCHID'})

In [30]:
merged_201718

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,American Indian/Alaska Native (% total),Asian (% total),Black/African American (% total),Hispanic/Latino (% total),Hawaiian/Pacific Islander (% total),Not Specified (% total),Two+ races (% total),White (% total),Free or Reduced Lunch (% total),Total Student Count,TEACHERS,TOTAL_INDICATOR,DMS_FLAG,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR
0,2017-2018,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,NaN,AL-101,100005,AL-101-0010,10000500870,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336.0,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,NaN,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,04/04/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,07,08,Middle,As reported,0.000000,0.619579,3.965304,45.600991,0.123916,0.0,2.230483,47.459727,70.631970,807.0,36.0,Education Unit Total,Reported,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.260194,-86.206174,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,104,01026,01009,2017-2018
1,2017-2018,1,ALABAMA,AL,Albertville High School,Albertville City,1,NaN,AL-101,100005,AL-101-0020,10000500871,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399.0,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322.0,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,04/04/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported,0.278552,0.557103,3.690808,42.757660,0.000000,0.0,2.855153,49.860724,57.938719,1436.0,77.0,Education Unit Total,Reported,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.262226,-86.204884,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,104,01026,01009,2017-2018
2,2017-2018,1,ALABAMA,AL,Evans Elementary School,Albertville City,1,NaN,AL-101,100005,AL-101-0110,10000500879,100879,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300.0,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1346.0,(256)878-7698,http://www.albertk12.org,1,Open,1,Open,04/04/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,05,06,Middle,As reported,0.117647,0.588235,2.823529,51.294118,0.000000,0.0,2.352941,42.823529,71.529412,850.0,36.0,Education Unit Total,Reported,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual,Evans Elementary School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.273318,-86.220144,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,104,01026,01009,2017-2018
3,2017-2018,1,ALABAMA,AL,Albertville Elementary School,Albertville City,1,NaN,AL-101,100005,AL-101-0200,10000500889,100889,1100 Horton Rd,NaN,NaN,Albertville,AL,35950,2532.0,1100 Horton Rd,NaN,NaN,Albertville

In [31]:
# Save final, cleaned data set
merged_201718.to_csv(data_fp + "ccd_201718/ccd_sch_merged_1718.csv", header = True)
merged_201718.to_excel(data_fp + "ccd_201718/ccd_sch_merged_1718.xlsx", header = True)

ValueError: I/O operation on closed file.